In [23]:
import numpy as np
import torchvision
import torch
from torchvision import datasets, transforms
import tqdm

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
data = datasets.MNIST(root="../vae/data/",
                       transform = transform,
                       train = False)
loader = torch.utils.data.DataLoader(dataset=data, batch_size = 10, shuffle = False)

In [12]:
class RBM():
    def __init__(self, n_input, n_hidden=128, learning_rate=0.1, n_iterations=100):
        self.n_input = n_input
        self.n_iterations = n_iterations
        self.lr = learning_rate
        self.n_hidden = n_hidden
        self._initialize_weights()

    def _initialize_weights(self):
        self.W = torch.randn(self.n_input, self.n_hidden)*0.1
        self.v0 = torch.zeros(1, self.n_input)       # Bias visible
        self.h0 = torch.zeros(1, self.n_hidden)   # Bias hidden

    def fit(self, loader):
        '''Contrastive Divergence training procedure'''

        self.training_errors = []
        for _ in tqdm.tqdm(range(self.n_iterations)):
            batch_errors = []
            for batch, _ in loader:
                batch = (batch > 0.5).float().reshape(-1, self.n_input)
                # Positive phase
                positive_hidden = torch.sigmoid(batch.mm(self.W) + self.h0)
                hidden_states = self._sample(positive_hidden)
                positive_associations = batch.T.mm(positive_hidden)

                # Negative phase
                negative_visible = torch.sigmoid(hidden_states.mm(self.W.T) + self.v0)
                negative_visible = self._sample(negative_visible)
                negative_hidden = torch.sigmoid(negative_visible.mm(self.W) + self.h0)
                negative_associations = negative_visible.T.mm(negative_hidden)

                self.W  += self.lr * (positive_associations - negative_associations)
                self.h0 += self.lr * (positive_hidden.sum(dim=0) - negative_hidden.sum(dim=0))
                self.v0 += self.lr * (batch.sum(dim=0) - negative_visible.sum(dim=0))

                batch_errors.append(((batch - negative_visible) ** 2).mean().item())

            self.training_errors.append(np.mean(batch_errors))

    def _sample(self, X):
        return (X > torch.rand_like(X)).float()

    def reconstruct(self, X):
        positive_hidden = torch.sigmoid(X.mm(self.W) + self.h0)
        hidden_states = self._sample(positive_hidden)
        negative_visible = torch.sigmoid(hidden_states.mm(self.W.T) + self.v0)
        return negative_visible

In [13]:
model = RBM(784, n_hidden=36, learning_rate=0.01, n_iterations=40)

In [14]:
model.fit(loader)

100%|███████████████████████████████████████████| 40/40 [00:52<00:00,  1.30s/it]


In [15]:
X = data.data.reshape(-1, 784)
X = (X > 0.5).float()
recon = model.reconstruct(X)